In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fr2g3d56
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fr2g3d56
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   

## Transcibe Example

In [2]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/content/Back to December.mp3")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 56.7MiB/s]


In [8]:
import re
text = re.sub(r"(.{40})", "\\1\r\n", result["text"])
print(text)

 I'm so glad you made time to see me hou
se life tell me as your family I haven't
 seen them in a while you've been good b
usier than ever your small talk working 
the weather your guard is up and I know 
why because the last time


In [16]:
model = whisper.load_model("small")
result = model.transcribe("/content/Back to December.mp3")
text = re.sub(r"(.{40})", "\\1\r\n", result["text"])
print(text)

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 62.3MiB/s]


 I'm so glad you made time to see me. Ho
w's life? Tell me how's your family? I h
aven't seen them in a while. You've been
 good, busier than ever. Your small talk
, work and the weather. God is up and I 
know why. Because the last time I saw yo
u was when I was a kid.


In [17]:
model = whisper.load_model("large")
result = model.transcribe("/content/Back to December.mp3")
text = re.sub(r"(.{40})", "\\1\r\n", result["text"])
print(text)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:63: UserWarning: /root/.cache/whisper/large-v2.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.87G/2.87G [00:52<00:00, 58.6MiB/s]


 I'm so glad you made time to see me. Ho
w's life? Tell me, how's your family? I 
haven't seen them in a while. You've bee
n good, busier than ever. Your small tal
k, work, and the weather. God is up and 
I know why. Because the last time...


## Low level Example

In [13]:
model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/content/How-You-Like-That.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
# - set the fp16 flag
options = whisper.DecodingOptions(fp16=False)
result = whisper.decode(model, mel, options)

Detected language: ko
